### Imports

In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import functions

In [33]:
spark = SparkSession \
    .builder \
    .appName("Lab7") \
    .getOrCreate()

context = spark.sparkContext

### Reading employees into the dataframe

In [34]:
schema = StructType([
    StructField("emp_id", StringType(), False),
    StructField("fname", StringType(), True),
    StructField("lname", StringType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("zipcode", StringType(), True),
    StructField("job_title", StringType(), True),
    StructField("email", StringType(), True),
    StructField("active", StringType(), True),
    StructField("salary", IntegerType(), True),
])

df = spark.read.format("csv") \
    .option("sep", "\t") \
    .option("header", True) \
    .schema(schema) \
    .load("./data/employees.txt")

df.createOrReplaceTempView("employees")

In [35]:
df.show()

+---------+--------+---------+--------------------+----------------+-----+-------+------------------+--------------------+------+------+
|   emp_id|   fname|    lname|             address|            city|state|zipcode|         job_title|               email|active|salary|
+---------+--------+---------+--------------------+----------------+-----+-------+------------------+--------------------+------+------+
|AA1130960|     Amy|   Alicea|    2472 25th Street|          Fulton|   KY|  42041|   Sales Associate|aa1130960@example...|     Y| 22329|
|AA1146303|    Anna|   Atkins|4640 East 10th St...|   Scotts Valley|   CA|  95066| Assistant Manager|aa1146303@example...|     Y| 99645|
|AA1154964|   Annie|Albritton|4444 West 14th St...|          Thorpe|   WV|  24888|   Sales Associate|aa1154964@example...|     Y| 26717|
|AA1352280| Antoine|  Aguirre|3293 West 2nd Street|          Mobile|   AL|  36630|   Sales Associate|aa1352280@example...|     Y| 26078|
|AA1411429|  Arthur| Andersen|    4216 21

### Exercise 1

#### Dataframe API/DSL

In [36]:
df.orderBy(functions.col("fname").asc(), functions.col("lname").desc()).limit(5).show()

+---------+-----+-------+--------------------+-----------+-----+-------+------------------+--------------------+------+------+
|   emp_id|fname|  lname|             address|       city|state|zipcode|         job_title|               email|active|salary|
+---------+-----+-------+--------------------+-----------+-----+-------+------------------+--------------------+------+------+
|AW2136153|Aaron|  Woody|      131 7th Street|     Mantua|   NJ|  08051|   Sales Associate|aw2136153@example...|     Y| 22164|
|AW1399370|Aaron|   Weil|4587 East 21st St...| Sacramento|   CA|  94232|           Cashier|aw1399370@example...|     Y| 17208|
|AW1743700|Aaron| Waters|32887 West 4th St...|      Ocala|   FL|  34475|   Sales Associate|aw1743700@example...|     Y| 20026|
|AV5801330|Aaron|Vincent|8714 North Foster...|  San Mateo|   CA|  94405|   Sales Associate|av5801330@example...|     Y| 28575|
|AV3024299|Aaron|Velasco|21925 West 7th St...|Watsonville|   CA|  95077|Maintenance Worker|av3024299@example...

#### SQL Language

In [37]:
spark.sql("SELECT * FROM employees ORDER BY fname ASC, lname DESC").show()

+---------+-----+---------+--------------------+------------+-----+-------+--------------------+--------------------+------+------+
|   emp_id|fname|    lname|             address|        city|state|zipcode|           job_title|               email|active|salary|
+---------+-----+---------+--------------------+------------+-----+-------+--------------------+--------------------+------+------+
|AW2136153|Aaron|    Woody|      131 7th Street|      Mantua|   NJ|  08051|     Sales Associate|aw2136153@example...|     Y| 22164|
|AW1399370|Aaron|     Weil|4587 East 21st St...|  Sacramento|   CA|  94232|             Cashier|aw1399370@example...|     Y| 17208|
|AW1743700|Aaron|   Waters|32887 West 4th St...|       Ocala|   FL|  34475|     Sales Associate|aw1743700@example...|     Y| 20026|
|AV5801330|Aaron|  Vincent|8714 North Foster...|   San Mateo|   CA|  94405|     Sales Associate|av5801330@example...|     Y| 28575|
|AV8865900|Aaron|  Velasco|3484 West 10th St...| Hustonville|   KY|  40437| 

### Exercise 2

#### Dataframe API/DSL

In [40]:
df.select("job_title").distinct().count()

76

#### SQL Language

In [46]:
spark.sql("SELECT COUNT(DISTINCT job_title) as Count FROM employees").show()

+-----+
|Count|
+-----+
|   76|
+-----+



### Exercise 3

#### Dataframe API/DSL

In [30]:
df.groupBy("job_title").agg(
        functions.count_distinct("emp_id").alias("# Employees"),
        functions.avg("salary").alias("Average Salary")
    ).show()

+--------------------+-----------+------------------+
|           job_title|# Employees|    Average Salary|
+--------------------+-----------+------------------+
|           President|          1|         3500000.0|
|Information Secur...|          7| 81376.85714285714|
|       Store Manager|        703| 47178.45234708393|
|      Fraud Examiner|          2|           67878.0|
|Business Intellig...|         13| 91869.92307692308|
|          Landscaper|         62| 27045.08064516129|
|      Vice President|         13|166443.38461538462|
|     Sales Associate|      39747|24276.333660402044|
|      Security Guard|        745|19270.107382550337|
|System Administrator|         32|       95883.21875|
|  Maintenance Worker|        730|25736.098630136985|
|                 CTO|          1|          695000.0|
|Investor Relation...|          1|           33500.0|
|             Cashier|      13621| 19582.02532853682|
|            Director|         23|135962.47826086957|
|   Marketing Manager|      

#### SQL Language

In [51]:
spark.sql("SELECT job_title, COUNT(DISTINCT emp_id) AS EmployeesCount, \
     AVG(salary) AS AverageSalary FROM employees GROUP BY job_title").show()

+--------------------+--------------+------------------+
|           job_title|EmployeesCount|     AverageSalary|
+--------------------+--------------+------------------+
|           President|             1|         3500000.0|
|Information Secur...|             7| 81376.85714285714|
|       Store Manager|           703| 47178.45234708393|
|      Fraud Examiner|             2|           67878.0|
|Business Intellig...|            13| 91869.92307692308|
|          Landscaper|            62| 27045.08064516129|
|      Vice President|            13|166443.38461538462|
|     Sales Associate|         39747|24276.333660402044|
|      Security Guard|           745|19270.107382550337|
|System Administrator|            32|       95883.21875|
|  Maintenance Worker|           730|25736.098630136985|
|                 CTO|             1|          695000.0|
|Investor Relation...|             1|           33500.0|
|             Cashier|         13621| 19582.02532853682|
|            Director|         